<h1><span style="color:red">Simple Variable Calculations</span></h1>

### This sample notebook reads numeric variables from a survey dataset and lets users compute a new numeric variable, add it to a new survey version, and publish the survey to the user's surveys gallery

## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

In [ ]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

pn.extension()
def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint


## 2. Read the survey file and extract numeric variables

In [ ]:
# read the csv file
df = panellibs.extract_data(absolutePath + csv_file)

# create a list of variable names
variables_df = pd.DataFrame({'varname':df.columns})
printmd("<b><span style='color:red'>All variables in the survey file:</span></b>")
print(variables_df.varname.values)

# create a dictionary of #number variables with abbreviated and full variable names 
var_list = {n[:n.index('#')]:n for n in variables_df.varname.values if '#number' in n}
printmd("<b><span style='color:red'>Numeric variables:</span></b>")
print(var_list)


## 3. Define a new variable using the form ##

In [ ]:
# Define a new Variable
a = widgets.Dropdown(options=var_list.keys())
b = widgets.Dropdown(options=['/','*','+','-'])
c = widgets.Dropdown(options=var_list.keys())
ui = widgets.VBox([a, b, c])
def f(a, b, c):
    return ((a, b, c))

formula = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, formula)

printmd("<b><span style='color:red'>Select variables and the operation, then run the next cell</span></b>")


## 4. Edit variable name as needed

In [ ]:
# Give a Name to the New Variable
def f(Var_Name):
    return Var_Name
newvar = interact(f, Var_Name=a.value + b.value + c.value +'#number');

printmd("<b><span style='color:red'>After defining variable name hit Enter, then run the next cell</span></b>")


## 5. Compute the new variable and format it for SuAVE

In [ ]:
try:
    if b.value == '/':
        df[newvar.widget.result] = df[var_list[a.value]] / df[var_list[c.value]]
    elif b.value == '*':
        df[newvar.widget.result] = df[var_list[a.value]] * df[var_list[c.value]]
    elif b.value == '+':
        df[newvar.widget.result] = df[var_list[a.value]] + df[var_list[c.value]]
    elif b.value == '-':
        df[newvar.widget.result] = df[var_list[a.value]] - df[var_list[c.value]]
        
# make sure there are no illegal NaN type values in this #number variable
    df[newvar.widget.result].fillna('',inplace=True)
    df.replace(['None', 'nan', np.inf, -np.inf], np.nan, inplace=True)
    df[newvar.widget.result] = pd.to_numeric(df[newvar.widget.result], errors='coerce', downcast='float')
    df[newvar.widget.result] = df[newvar.widget.result].apply(lambda x: '{:.6f}'.format(x))
    df.replace('nan', "", inplace=True)
    printmd("<b><span style='color:red'>New variable computed</span></b>")

except:
    printmd("<b><span style='color:red'>!! Cannot compute. One or both variables contain non-numeric values!!</span></b>")
    printmd("<b>" + var_list[a.value] +"</b>")
    print(df[var_list[a.value]].describe())
    printmd("<b>" + var_list[c.value] +"</b>")
    print(df[var_list[c.value]].describe())

panellibs.slider(df)

## 6. Save the new version of CSV file, and give a name to new survey

In [ ]:
new_file = suaveint.save_csv_file(df, absolutePath, csv_file)

In [ ]:
#Input survey name

from IPython.display import display
input_text = widgets.Text(placeholder='Enter Survey Name...')
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)


In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)


## 7. Generate the survey and create survey URL

In [ ]:
suaveint.create_survey(survey_url,new_file, survey_name, dzc_file, user, csv_file, view, views)